# TimeStamp in ``pyarrow``

## 时间戳的本质

**计算机世界中如何表达时间?**

在人类的语言中, 我们通常用 ``2001-01-15 08:30:43.000_000_001T00:00`` (精确到1ns) 这样的字符串来描述时间. 但在计算机的世界里, 用字符串来保存时间是非常低效的. 这是因为即使是 ASCII 字符串, 一个字符也要占用 8 bit. 这么长一串字符串, 如果要提供 ns 级别的精度, 即使不考虑时区, 也需要 ``len("2001-01-15 08:30:43.000000001") * 8`` 232 bit. 非常不利于存储. 并且字符串也很不便于计算, 比如你想给一个时间加 1 天零 2 个小时, 就要考虑很多进位的问题, 非常不变.

在计算机的世界里, 通常用 64 位整数来表示时间. 一般我们默认一个时间起点, 业内通常用 Unix EPOCH (unix 系统的发明时间) 也就是 ``1970-01-01 00:00:00`` 作为起点 (也有把 BC, 也就是公元 0000 年, 作为起点的, 不过很少见), 然后把某个时间到 EPOCH 的纳秒数用整数表示, 这个整数就可以用来唯一确定一个时间了. 我们可以简单做一个计算:

- 32 bit 可以表示的最大整数为 2 ** 32 = 4,294,967,296
- 64 bit 可以表示的最大整数为 2 ** 64 = 18,446,744,073,709,551,616
- 从 EPOCH 开始, 如果精度是 1 秒, 32 位整数可以表示到 2106-02-07 06:28:16
- 从 EPOCH 开始, 如果精度是 1 毫秒, 32 位整数可以表示到 1970-02-19 17:02:47.296
- **从 EPOCH 开始, 如果精度是 1 纳秒, 64 位整数可以表示到 2554-07-21 23:34:33.709553 (从 EPOCH 起 584 年), 足够我们用了**. 如果精度是 1 微秒 则从 EPOCH 起能用 584,000 年, 这完全够用了.

术语:

- EPOCH: 1970-01-01 00:00:00
- BC: 0000-01-01 00:00:00
- 毫秒 ms: 0.001 秒
- 微秒 microsecond: 0.000001 秒
- 纳秒 ns: 0.000000001 秒

In [19]:
from datetime import datetime, timedelta

int32 = 2 ** 32
int64 = 2 ** 64
epoch = datetime(1970, 1, 1)
print(epoch + timedelta(seconds= (int32 / 1)))
print(epoch + timedelta(seconds= (int32 / 1000)))
print(epoch + timedelta(seconds= (int64 / 1000000000)))

2106-02-07 06:28:16
1970-02-19 17:02:47.296000
2554-07-21 23:34:33.709553


## Timestamp in different System

在 datetime.datetime, pyarrow, pandas, spark 中, 管理时间戳数据的方法是不同的.

- datetime: 是精确到 microsecond 的 int64, 不支持 ns
- pyarrow: 是精确到 ns 的 int64
- pandas: 是可以精确到 ns 的 int64, 精度可调
- spark: 是可以精确到 microseconds 的 int64, 不支持 ns

这里要注意的是 spark 的时间戳对象是不带 timezone 信息的, 而是用 spark session 当前的 timezone. 不过你可以用 ``spark.conf.set("spark.sql.session.timeZone", "UTC")`` 来修改 spark session 的 timeazone (一般是设置为 UTC)

Ref:

- pyarrow timestamp: https://arrow.apache.org/docs/python/timestamps.html
- padnas timestamp: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html


In [1]:
from datetime import datetime
import pandas as pd
import pyarrow as pa

In [2]:
datetime_list = [
    datetime(2000, 1, 1), 
    datetime(2000, 1, 2),
    datetime(2000, 1, 3),
]

t = pa.table({"time": datetime_list})
df = pd.DataFrame({"time": datetime_list})

In [3]:
t

pyarrow.Table
time: timestamp[us]
----
time: [[2000-01-01 00:00:00.000000,2000-01-02 00:00:00.000000,2000-01-03 00:00:00.000000]]

In [4]:
t["time"].type

TimestampType(timestamp[us])

In [5]:
df

,time
0,2000-01-01
1,2000-01-02
2,2000-01-03


In [6]:
df["time"].dtype

dtype('<M8[ns]')

In [8]:
pa.Table.from_pandas(df)["time"]

[
  [
    2000-01-01 00:00:00.000000000,
    2000-01-02 00:00:00.000000000,
    2000-01-03 00:00:00.000000000
  ]
]

In [9]:
t.to_pandas()["time"]

0   2000-01-01
1   2000-01-02
2   2000-01-03
Name: time, dtype: datetime64[ns]